In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz', '/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_tem

In [8]:
def tanh_simple(x,x0,K,theta):
    return K*(1+tanh(theta*(x-x0)))
def tanh_plus(x,x0,K,theta,a,b):
    return K*(1+tanh(theta*(x-x0))) + a*(x-x0)+b

def gen_logi(x,A,K,C,B,M,nu):
    return A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu)

def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func2ter(t, t0, A, B, D):
    return A*(t-t0)**(2*D) / (1+(t-t0)**2)**D + B
#def func2bis_to10(t, t0, A, B,  C, D):
#    tau = t-t0
#    return 10**( (A-B)*(tau) / (C+(tau**2)**(D/2))**(1/D) + B )
def func3(t, t0, A, B,  C):
    return A*(t-t0) / (C+(t-t0)**2) + B
def func4(t, t0, A, B, C):
    return A*(t-t0) / (C+abs(t-t0)) + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B
def func6(t, t0, A, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k)
def func7(t, t0, A, B, k):
    return A*(t-t0) / (1+abs(t-t0)**k)**(1/k) + B

In [9]:
to_fit = 22 #11
x  = time[to_fit]
y  = T_aux_avg[to_fit]
figure('testfitfunctions',clear='True')
#xlim(0.8,6)

#plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(x*1e3, y_filt+10,label=r'$T filt$',color='C1',ls=':')
vlines(x[end_fit]*1e3,1e-2,150,color='k')


#p0 = [t0_first_guess,2*10**3,0,1]
#plot(x[start_for_fit:end_fit]*1e3,func7_to10(x[start_for_fit:end_fit],*p0),color='chartreuse',label=r'$|t-t_0|^k, C=1$')

grid()
#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(13,15)
#ylim(-2,60)


NameError: name 'end_fit' is not defined

In [404]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_

In [406]:
# OK pour la temperature
to_plot = 12
end_fit = len(time[to_plot])//8-150 # 125500

print(file_path[to_plot])
# test plot
index_max = 1900
figure(1,clear='True')
#semilogy(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')
plot(time[to_plot]*1e3,T_aux_avg[to_plot],color='r')

#p0 = [1.8, 60, 63.45, 0.002,4e-3] # 124850
popt, pcov = curve_fit(tanh_simple,
                       time[to_plot][:end_fit]*1e3, log10(T_aux_avg[to_plot][:end_fit]),
                       maxfev = 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,tanh_simple(time[to_plot][:end_fit]*1e3,*popt),
     color='k')

#plot(time[to_plot][end_fit:]*1e3,tanh_plus_to10(time[to_plot][end_fit:]*1e3,
#                                            1635.3,
#                                            14.075,
#                                            63.132,
#                                            0.87,
#                                            -3324.94),
#     color='g',ls=':')


#p0 = [5e-3,500,1,2,65,2] # 124850
popt, pcov = curve_fit(gen_logi,
                       time[to_plot][:end_fit]*1e3, log10(T_aux_avg[to_plot][:end_fit]),
                       maxfev = 50000)
print(popt)
plot(time[to_plot][:end_fit]*1e3,gen_logi(time[to_plot][:end_fit]*1e3,*popt),
     color='xkcd:azul')

p0 = [1.95*1e-3,log10(500)/2,0.1,1e-1,0.3]
popt, pcov = curve_fit(func5, x[:end_fit], log10(y[:end_fit]), p0, maxfev= 50000)
print(popt)
y_fit = func5(x[:end_fit],*popt)
plot(x[:end_fit]*1e3,10**y_fit,color='g')
plot(x[:end_fit]*1e3,10**func5(x[:end_fit],1.95*1e-3,log10(500),0.1,1e-1,0.3),
     color='xkcd:rouge')

vlines(time[to_plot][end_fit]*1e3,0,50)
#b, a = signal.butter(8, 0.125)
#y = signal.filtfilt(b, a, T_aux_avg[to_plot], padlen=150)
#semilogy(time[to_plot]*1e3,y,color='b')
xlabel('time [ms]')
ylabel('T [K]')
#xlim(61,64)
#ylim(-1,20)
grid()
tight_layout()

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC06_RF06.npz
[-2.80274141e+02 -7.01651070e+01 -7.33856344e-03]


/tmp/ipykernel_184041/2210644391.py:7: RuntimeWarning: overflow encountered in power
  return A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu)


[-6.04263167 -2.01310632  1.00387135  0.96321514 -4.94272902  8.5009146 ]
[ 1.45413564e-03  1.38134073e-01 -2.07795199e+00  2.11476108e-02
  5.92726444e-01]


In [629]:
t0_first_guess,A0,B0

(0.013997624500000002, 41.99381961231696, 0.517953442987538)

In [410]:
y[start_for_fit:end_fit]

array([], dtype=float64)

In [673]:
to_fit = 7 #11
start_for_fit = 10
end_for_fit = [int(500000-1),140000,17700,7000,4800, # 17700
               470000,480000,2450,27500,5500, # 24000
               50000,28500,23500,30000,14500,
              0,0,0,0,0,
              0,0,27000,0,0,
              0,0]
end_fit = end_for_fit[to_fit]
x  = time[to_fit]
y  = T_aux_avg[to_fit]
T_crit = 0.5
t0_first_guess = x[where(min(abs(y-T_crit))==abs(y-T_crit))][0] # 2.5e-3
A0 = log10(max(y[start_for_fit:end_fit]))
B0 = log10(y[where(min(abs(y-T_crit))==abs(y-T_crit))][0])
figure('testfitinlog',clear='True')
#xlim(0.8,6)
grid()
semilogy(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
semilogy(x*1e3, y_filt ,label=r'$T filt$',color='k',ls=':',lw=3)
vlines(x[end_fit]*1e3,1e-2,150,color='k')

'''
p0 = [t0_first_guess,A0,B0,1e-3,0.4]
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit], log10(y_filt[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func5',popt)
kfunc5 = popt[4]
y_fit5 = func5(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'func5')

p0 = [t0_first_guess,A0,B0,0.05,0.4]
popt2, pcov = curve_fit(func2, x[start_for_fit:end_fit], log10(y_filt[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func2',popt)
y_fit2 = func2(x[start_for_fit:end_fit],*popt2)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2,color='m',label=r'func2')
perr2 = np.sqrt(np.diag(pcov))
'''
'''
p0 = [t0_first_guess,A0,B0,0.05,0.45]
popt, pcov = curve_fit(func2bis, x[start_for_fit:end_fit], log10(y_filt[start_for_fit:end_fit]), p0, maxfev= 100000)
print('func2bis',popt)
kfunc2bis = popt[4]
y_fit2bis = func2bis(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit2bis,color='orange',ls=':',label=r'func2bis')
perr2bis = np.sqrt(np.diag(pcov))
'''
'''
p0 = [t0_first_guess,log10(100),1,1,log10(0.1)]
popt, pcov = curve_fit(tanh_plus, x[start_for_fit:end_fit], log10(y_filt[start_for_fit:end_fit]), p0, maxfev= 50000)
print('tanh_simple',popt)
y_fittanh = tanh_plus(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fittanh,color='chartreuse',ls='-',label=r'tanh')

p0 = [t0_first_guess,A0,B0,0.001]
popt, pcov = curve_fit(func4, x[start_for_fit:end_fit], log10(y[start_for_fit:end_fit]), p0,  maxfev= 50000)
y_fit4 = func4(x[start_for_fit:end_fit],*popt)
print('func4',popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit4,color='g',ls='--',label=r'$|x|$')
'''
'''
p0 = [t0_first_guess,A0,0.05,0.05]
popt, pcov = curve_fit(func6, x[start_for_fit:end_fit], log10(y_filt[start_for_fit:end_fit]), p0, maxfev= 50000)
print('func6',popt)
y_fit6 = func6(x[start_for_fit:end_fit],*popt)
semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit6,color='r',ls=':',label=r'$|x|^k$')
'''

#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(1.7,2.3)
ylim(8e-4,120)
xlabel('time [ms]')
ylabel('Temp [K]')
tight_layout()

func2bis [ 1.80774027e-03  2.57773524e+00 -3.02554923e-01  9.10802316e-03
  3.37204569e-01]


In [649]:
print(perr2)
print(perr2bis)

[1.87452044e-07 2.58802874e-02 5.56797250e-02 9.77561395e-13
 1.09557827e-03]
[1.76120806e-04 4.22847727e+03 4.22846309e+03 6.45935540e+05
 8.53425162e-01]


In [451]:
figure('testfitinlin',clear='True')
#xlim(0.8,6)
grid()
plot(x*1e3, y ,label=r'$T$',color='C0')
b, a = signal.butter(8, 0.125)
y_filt = signal.filtfilt(b, a, y, padlen=150)
plot(x*1e3, y_filt ,label=r'$T filt$',color='C1',ls=':')
vlines(x[end_fit]*1e3,1e-2,150,color='k')

# plot(x[start_for_fit:end_fit]*1e3,10**y_fit4,color='r',ls=':',label=r'$|x|$')

plot(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$|x|^k$')

plot(x[start_for_fit:end_fit]*1e3,10**y_fit2,color='m',label=r'$(t-t_0)^2$')

plot(x[start_for_fit:end_fit]*1e3,10**y_fit2bis,color='orange',label=r'$(t-t_0)^{2D/2}$')

plot(x[start_for_fit:end_fit]*1e3,10**y_fittanh,color='g',ls='-',label=r'tanh')

#hlines(y[],1000,x[-1],color='k',ls=':')
legend()
#xlim(0.8,x[end_fit]*1e3+0.1)
#xlim(1.7,2.3)
#ylim(8e-4,120)

In [603]:
shape(x[start_for_fit:end_fit-1])

(974,)

In [624]:
DeltaF = 19
DeltaV = 7.2

print(DeltaF/DeltaV*1e-3 * 1e3,'MHz')

2.638888888888889 MHz


In [704]:
(Omega/2/pi)*1.8e-3

3689.9999999999995

In [709]:
time_step = 48//2 # 488 # 488
y_filt_diff = y_filt[time_step:]-y_filt[:-time_step]
x_avg = (x[:-time_step]) *1e3 # x_avg = (x[time_step:]+x[:-time_step])/2 *1e3
N_period = (x[time_step]-x[0])*(Omega/2/pi)

figure(11,clear='True')
plot(x_avg*(Omega/2/pi),y_filt_diff/N_period,color='k',ls=':',lw=3,label='Difference derivate')
plot(x[start_for_fit:end_fit-1]*1e3*(Omega/2/pi),
     diff(y_fit2)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='m',label=r'func2') # / 10000000
#plot(x[start_for_fit:end_fit-1]*1e3,
#     diff(y_fit2bis)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='orange' ) # / 10000000
plot(x[start_for_fit:end_fit-1]*1e3*(Omega/2/pi),
     diff(y_fit5)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='r',ls=':',label=r'func5') # / 10000000

xlabel('t [period]')
ylabel('H [K/period]')
legend()
grid()
#xlim(t0_first_guess*1e3-0.2,t0_first_guess*1e3+1)
xlim(3.6e6,4e6)
ylim(-0.051,0.35)
tight_layout()

4.878048780487805e-07

In [730]:
# as function of temperature

c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
tau_RF = 1/(Omega/(2*pi))
dT_dt = -1/tau*sqrt(m_Ca*y_filt[:-time_step]/kb)*h_pl_bar*k/m_Ca

time_step = 48//2 # 488 # 488
y_filt_diff = y_filt[time_step:]-y_filt[:-time_step]
x_avg = (x[:-time_step]) *1e3 # x_avg = (x[time_step:]+x[:-time_step])/2 *1e3
N_period = (x[time_step]-x[0])*(Omega/2/pi)

figure(11,clear='True')
plot(y_filt[:-time_step],y_filt_diff/N_period,color='k',ls=':',lw=3,label='Difference derivate')
plot(y_filt[:-time_step],-dT_dt*tau_RF,color='b',ls=':',lw=3,label='Cooling power')
plot(y_filt[:len(y_fit2)-1],
     diff(y_fit2)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='m',label=r'func2') # / 10000000
#plot(x[start_for_fit:end_fit-1]*1e3,
#     diff(y_fit2bis)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='orange' ) # / 10000000
plot(y_filt[:len(y_fit5)-1],
     diff(y_fit5)/(diff(x[start_for_fit:end_fit])*(Omega/2/pi)),color='r',ls=':',label=r'func5') # / 10000000

xlabel('T [K]')
ylabel('H [K/period]')
legend()
grid()
#xlim(t0_first_guess*1e3-0.2,t0_first_guess*1e3+1)
xlim(-0.2,10)
ylim(-0.05,0.35)
tight_layout()

In [514]:
# effect of C in func2
cm = pylab.get_cmap('jet')
xtest=linspace(0,20e-3,1000)*(Omega/2/pi)

print('test')
figure('test C in func2',clear='True')
i = 0
minC = -10
maxC = -5
for powC in range(minC,maxC):
    C = 10**(-powC)
    print(powC)
    pfit = [5e-3*(Omega/2/pi),log10(100),log10(1e-1),C,0.45]
    semilogy(xtest*1e3,10**func2(xtest,*pfit),color=cm(i/(maxC-minC-1)),label=f'C={powC}')
    i=i+1
grid()
legend()
tight_layout()

test
-10
-9
-8
-7
-6


In [544]:
# effect of C in func2
cm = pylab.get_cmap('jet')
xtest=linspace(0,20e-3,1000)*(Omega/2/pi)

print('test')
figure('test C in func5',clear='True')
i = 0
minC = -3
maxC = 2
for powC in range(minC,maxC):
    C = 10**(-powC)
    print(powC)
    pfit = [5e-3*(Omega/2/pi),log10(100),log10(1e-1),C,0.45]
    semilogy(xtest*1e3,10**func5(xtest,*pfit),color=cm(i/(maxC-minC-1)),label=f'C={powC}')
    i=i+1
grid()
legend()
tight_layout()

func5

test
-3
-2
-1
0
1


<function __main__.func5(t, t0, A, B, C, k)>

In [569]:
log10(0.01e-1)

-3.0

In [574]:
# effect of C in func2
cm = pylab.get_cmap('jet')
xtest=linspace(0,20e-3,50000)*(Omega/2/pi)

print('test')
figure('test k in func5',clear='True')
i = 0
powk = linspace(0.001,0.01,10)
for powk_loop in powk:
    #[ 1.807e-03, 2.386e+00,-3.072e-01, 6.085e-03, 3.752e-01]
    pfit = [1.8e-3*(Omega/2/pi),log10(100),log10(1e-1),6e-3,powk_loop]
    plot(xtest*1e3,10**func5(xtest,*pfit),color=cm(i/(maxC-minC-1)),label=f'C={powk_loop:.2f}')
    i=i+1
grid()
legend(title = 'powk')
tight_layout()

func5

test


<function __main__.func5(t, t0, A, B, C, k)>